In [1]:
#import the neccessary libraries 
import csv
import pandas as pd
import requests
import json
from pprint import pprint 

# code necessary incase you'd like to search for a specific artist and their top 5 songs

artist = 'term=polyphia'
media = 'media=music'
entity = 'entity=song'
limit = 'limit=5'

url = 'http://itunes.apple.com/search?' + artist + media + entity + limit

In [2]:
#list of countries we narrow down to sort through our data

countries = ['hk', 'nl', 'ru', 'au', 'ca', 'fr', 'jp', 'gb', 'de', 'us' ]

In [3]:
#create a dataframe of the current top 5 songs in the US

#set up api url and output the response data

url = "https://rss.itunes.apple.com/api/v1/us/apple-music/top-songs/all/5/explicit.json"
response = requests.get(url).json()

#create lists that will take in the name, album, artist, genre, and rank of each song
song = []
album = []
artist = []
genre = []
rank = []

counter = 0

for i,t in enumerate (response["feed"]["results"]):
    name_1=(i,t["name"])
    album_1=(t["collectionName"])
    song.append(name_1[1])
    album.append(album_1)
    artist.append(t["artistName"])
    genre.append(t["genres"][0]["name"])
    rank.append(counter + 1)
    
    counter += 1

#use the data from above to create a dataframe of the top 5 songs
    
us = {
     "Name": song,
    "Album": album,
    "Artist": artist,
    "Genre" : genre,
    "Rank" : rank
}

us_df = pd.DataFrame(us)
us_df.set_index("Rank")

,Name,Album,Artist,Genre
Rank,,,,
1,Heartless,Heartless - Single,The Weeknd,R&B/Soul
2,Woah,Woah - Single,Lil Baby,Hip-Hop/Rap
3,Blinding Lights,Blinding Lights - Single,The Weeknd,R&B/Soul
4,BOP,KIRK,DaBaby,Hip-Hop/Rap
5,ROXANNE,ROXANNE - Single,Arizona Zervas,Hip-Hop/Rap


In [15]:
#create a dataframe similar to above that takes in all the top 5 songs from each country on our list

song_list = []
artist_list = []
album_list = []
country_list = []
genre_list = []
rank = []

for country in countries:
    top_list = []
    music_url = f"https://rss.itunes.apple.com/api/v1/{country}/apple-music/top-songs/all/5/explicit.json"
    
    try:
        response = requests.get(music_url).json()
        top_list.append(response["feed"]["results"])
        country_df = pd.DataFrame(top_list)
        for x in range (0,5):
            rank.append(x+1)
            song_list.append(country_df[x][0]['name'])
            country_list.append(country)
            artist_list.append(country_df[x][0]['artistName'])
            album_list.append(country_df[x][0]["collectionName"])
            genre_list.append(country_df[x][0]["genres"][0]["name"])
        
    except:
        print(f"Can't find {country}. Skipping... ")

top5_df = pd.DataFrame({
    "Country" : country_list,
    "Name" : song_list,
    "Artist" : artist_list,
    "Album" : album_list,
    "Genre" : genre_list,
    "Rank" : rank
})

top5_df['Country'] = top5_df['Country'].replace({'hk': 'Hong Kong',
                           'nl': 'Netherlands',
                           'ru': 'Russia',
                           'au': 'Australia',
                           'ca': 'Canada',
                           'fr': 'France',
                           'jp': 'Japan',
                           'gb': 'UK',
                           'de': 'Denmark',
                           'us': 'US'
                          })
top5_df = top5_df.set_index("Rank")
top5_df

,Country,Name,Artist,Album,Genre
Rank,,,,,
1,Hong Kong,Señorita,Shawn Mendes & Camila Cabello,Señorita - Single,流行樂
2,Hong Kong,Into the Unknown,Idina Menzel & AURORA,Frozen 2 (Original Motion Picture Soundtrack /...,原聲帶
3,Hong Kong,說好不哭,周杰倫 & 阿信,說好不哭 - Single,國語流行樂
4,Hong Kong,bad guy,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",另類音樂
5,Hong Kong,句號,G.E.M. 鄧紫棋,句號 - Single,國語流行樂
1,Netherlands,Dance Monkey,Tones and I,Dance Monkey - Single,Alternative
2,Netherlands,everything i wanted,Billie Eilish,everything i wanted - Single,Alternative
3,Netherlands,Blinding Lights,The Weeknd,Blinding Lights - Single,R&B/Soul
4,Netherlands,Memories,Maroon 5,Memories - Single,Pop


In [16]:
#import spotify libraries
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy.oauth2 as oauth2

NADA_CLIENT_ID='435b32dd35304b11b6ab03d8463dd2bd'
NADA_CLIENT_SECRET='a86f35f553bf4b69a25ea28f6ea890b6'

In [17]:
#get authentication from spotify's api 
spotify = spotipy.Spotify()
credentials = oauth2.SpotifyClientCredentials(
        client_id = NADA_CLIENT_ID,
        client_secret = NADA_CLIENT_SECRET)

token = credentials.get_access_token()
spotify = spotipy.Spotify(auth=token)

#extract Spotify ID from the top songs dataframe above , note spotify sometimes doesn't include features in the song name

spotify_id = []
for index, row in top5_df.iterrows():
    try:
        response = spotify.search(row["Name"], type="track", limit=1)
        spotify_id.append(response["tracks"]["items"][0]["album"]["artists"][0]["id"])
    except:
        text = row["Name"]
        head, sep, tail = text.partition('(')
        response = spotify.search(head, type="track", limit=1)
        spotify_id.append(response["tracks"]["items"][0]["album"]["artists"][0]["id"])
        continue
        
top5_df["Spotify ID"] = spotify_id
top5_df

,Country,Name,Artist,Album,Genre,Spotify ID
Rank,,,,,,
1,Hong Kong,Señorita,Shawn Mendes & Camila Cabello,Señorita - Single,流行樂,7n2wHs1TKAczGzO7Dd2rGr
2,Hong Kong,Into the Unknown,Idina Menzel & AURORA,Frozen 2 (Original Motion Picture Soundtrack /...,原聲帶,0LyfQWJT6nXafLPZqxe9Of
3,Hong Kong,說好不哭,周杰倫 & 阿信,說好不哭 - Single,國語流行樂,2elBjNSdBE2Y3f0j1mjrql
4,Hong Kong,bad guy,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",另類音樂,6qqNVTkY8uBg9cP3Jd7DAH
5,Hong Kong,句號,G.E.M. 鄧紫棋,句號 - Single,國語流行樂,7aRC4L63dBn3CiLDuWaLSI
1,Netherlands,Dance Monkey,Tones and I,Dance Monkey - Single,Alternative,2NjfBq1NflQcKSeiDooVjY
2,Netherlands,everything i wanted,Billie Eilish,everything i wanted - Single,Alternative,6qqNVTkY8uBg9cP3Jd7DAH
3,Netherlands,Blinding Lights,The Weeknd,Blinding Lights - Single,R&B/Soul,1Xyo4u8uXC1ZmMpatF05PJ
4,Netherlands,Memories,Maroon 5,Memories - Single,Pop,04gDigrS5kc9YWfZHwBETP


In [21]:
from googletrans import Translator 
translator = Translator()

top5_df['Translated Genre'] = top5_df['Genre'].apply(translator.translate, dest='en').apply(getattr, args=('text',))
top5_df['Source'] = 'Apple'
top5_df['Translated Genre'] = top5_df['Translated Genre'].replace({'pop music' : 'Pop', 
                                                                   'Mandarin pop music' : 'Pop',
                                                                   'Alternative Music' : 'Alternative',
                                                                   'R & B, soul' : 'R&B/Soul'
                                                                })
top5_df

,Country,Name,Artist,Album,Genre,Spotify ID,Translated Genre,Source
Rank,,,,,,,,
1,Hong Kong,Señorita,Shawn Mendes & Camila Cabello,Señorita - Single,流行樂,7n2wHs1TKAczGzO7Dd2rGr,Pop,Apple
2,Hong Kong,Into the Unknown,Idina Menzel & AURORA,Frozen 2 (Original Motion Picture Soundtrack /...,原聲帶,0LyfQWJT6nXafLPZqxe9Of,Soundtrack,Apple
3,Hong Kong,說好不哭,周杰倫 & 阿信,說好不哭 - Single,國語流行樂,2elBjNSdBE2Y3f0j1mjrql,Pop,Apple
4,Hong Kong,bad guy,Billie Eilish,"WHEN WE ALL FALL ASLEEP, WHERE DO WE GO?",另類音樂,6qqNVTkY8uBg9cP3Jd7DAH,Alternative,Apple
5,Hong Kong,句號,G.E.M. 鄧紫棋,句號 - Single,國語流行樂,7aRC4L63dBn3CiLDuWaLSI,Pop,Apple
1,Netherlands,Dance Monkey,Tones and I,Dance Monkey - Single,Alternative,2NjfBq1NflQcKSeiDooVjY,Alternative,Apple
2,Netherlands,everything i wanted,Billie Eilish,everything i wanted - Single,Alternative,6qqNVTkY8uBg9cP3Jd7DAH,Alternative,Apple
3,Netherlands,Blinding Lights,The Weeknd,Blinding Lights - Single,R&B/Soul,1Xyo4u8uXC1ZmMpatF05PJ,R&B/Soul,Apple
4,Netherlands,Memories,Maroon 5,Memories - Single,Pop,04gDigrS5kc9YWfZHwBETP,Pop,Apple


In [22]:
output_file = top5_df.to_csv ('appleAPI_top5_songs_per_country.csv', index = None, header=True)